# 실습 11: 전체 시스템 테스트 & QnA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/main/notebooks/Lab11_전체테스트.ipynb)

## 🎯 학습 목표
- 전체 파이프라인 통합 테스트
- 실무 시나리오 적용
- 성능 평가 및 최적화

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 전체 환경 재설치
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git
!apt-get install -y tesseract-ocr tesseract-ocr-kor

print("✅ 환경 설정 완료!")


In [ ]:
# 통합 Document AI 시스템
from docai_course.ocr import OCRBenchmark
from docai_course.preprocessing import DocumentPreprocessor
from docai_course.ner import UnifiedNERSystem
from docai_course.hybrid import HybridDocumentAI

# 시스템 초기화
system = HybridDocumentAI(
    use_preprocessing=True,
    use_ensemble=True,
    use_llm=False  # API 키 없으면 False
)

print("✅ 하이브리드 Document AI 시스템 준비 완료!")


In [ ]:
# 복잡한 실전 문서 생성
from PIL import Image, ImageDraw
import cv2
import numpy as np

img = Image.new('RGB', (900, 600), color='white')
draw = ImageDraw.Draw(img)

document = """
=========================================
          주식회사 테크노베이션
          프리랜서 용역 계약서
=========================================

1. 계약 정보
   - 계약번호: FR-2025-0042
   - 계약일자: 2025년 1월 20일
   - 계약기간: 2025.02.01 ~ 2025.07.31

2. 발주처 (갑)
   - 상호: 주식회사 테크노베이션
   - 대표이사: 박민수
   - 사업자번호: 123-45-67890
   - 주소: 서울시 강남구 테헤란로 123
   - 담당자: 김철수 팀장
   - 연락처: 02-1234-5678
   - 이메일: chulsoo.kim@technovation.com

3. 수주처 (을)
   - 성명: 이영희
   - 주민번호: 850315-2******
   - 주소: 경기도 성남시 분당구 정자동
   - 연락처: 010-9876-5432
   - 이메일: younghee.lee@gmail.com

4. 계약 금액
   - 계약금액: 총 18,000,000원 (VAT 별도)
   - 지급조건: 월말 기준 익월 15일 지급
   - 입금계좌: 국민은행 123-456-789012

5. 업무 내용
   - AI 기반 문서 처리 시스템 개발
   - OCR 엔진 최적화 및 성능 튜닝
   - 문서 분류 모델 학습 및 배포
"""

draw.text((30, 20), document, fill='black')

# 노이즈와 기울기 추가 (현실적인 스캔 문서)
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
noise = np.random.normal(0, 15, img_cv.shape).astype(np.uint8)
img_cv = cv2.add(img_cv, noise)

# 기울기
rows, cols = img_cv.shape[:2]
M = cv2.getRotationMatrix2D((cols/2, rows/2), 2, 1)
img_cv = cv2.warpAffine(img_cv, M, (cols, rows), borderValue=(255,255,255))

cv2.imwrite('complex_contract.jpg', img_cv)
print("✅ 복잡한 실전 문서 생성 완료!")


In [ ]:
# 전체 파이프라인 실행
import time

start_time = time.time()

print("\\n" + "="*70)
print("🚀 Document AI 전체 파이프라인 실행")
print("="*70)

# 1단계: 전처리
print("\\n[1/4] 📸 이미지 전처리 중...")
preprocessor = DocumentPreprocessor(debug_mode=False)
processed_img, binary_img, quality = preprocessor.adaptive_preprocessing_pipeline(
    'complex_contract.jpg'
)
print(f"     품질 점수: {quality.get('overall_score', 'N/A')}")

# 2단계: OCR
print("\\n[2/4] 📄 OCR 실행 중...")
benchmark = OCRBenchmark()
ocr_results = benchmark.test_tesseract(binary_img)
full_text = ocr_results['text']
print(f"     추출된 텍스트 길이: {len(full_text)} 자")

# 3단계: NER
print("\\n[3/4] 🔍 정보 추출 중...")
ner = UnifiedNERSystem()
entities = ner.rule_based_ner(full_text)
print(f"     추출된 엔티티: {len(entities)}개")

# 4단계: 구조화
print("\\n[4/4] 📊 데이터 구조화 중...")
import pandas as pd
entity_df = pd.DataFrame([
    {'타입': e['entity'], '값': e['text'], '신뢰도': f"{e['confidence']*100:.0f}%"}
    for e in entities
])

processing_time = time.time() - start_time

print("\\n" + "="*70)
print("✅ 처리 완료!")
print("="*70)
print(f"\\n총 처리 시간: {processing_time:.2f}초\\n")

print("📋 추출된 핵심 정보:")
print(entity_df.to_string(index=False))

# 결과 저장
entity_df.to_csv('final_result.csv', index=False, encoding='utf-8-sig')
print("\\n💾 결과 저장: final_result.csv")


## 🎓 수료 체크리스트

강의를 통해 배운 내용을 체크해보세요!

### 기초 지식
- [ ] Document AI의 전체 기술 스택 이해
- [ ] OCR 엔진별 특징과 선택 기준
- [ ] 신뢰도(Confidence) 의미와 활용

### 기술 습득
- [ ] PaddleOCR, Tesseract 등 OCR 엔진 사용
- [ ] 이미지 전처리로 정확도 향상
- [ ] NER을 통한 정보 추출
- [ ] 표 검출 및 구조화

### 고급 기법
- [ ] 다중 OCR 앙상블
- [ ] 적응적 전처리 파이프라인
- [ ] 하이브리드 Document AI 시스템

### 실무 역량
- [ ] 엔드투엔드 파이프라인 구축
- [ ] 성능 평가 및 최적화
- [ ] 실전 문서 처리

---

## 📞 질문 & 답변

강의 중 궁금한 점이 있으시면 GitHub Issues에 남겨주세요!

**Repository**: https://github.com/leecks1119/document_ai_lecture
